Необходимо запустить практическую часть занятия- файл находиться в материалых, и посмотреть самому то, о чём говорили на лекции.
По образу практики, попробуйте создать искусственный датасет с лишними столбцами.
Целевую метку, при правильной обработке данных, формируйте таким образом, чтобы без затруднений её смогла описать линейная модель.
Ориентируйтесь на то, что было показано во время занятия, и каждый шаг описывайте в markdown.
Здесь важно видеть ваш ход мысли. Не бойтесь ошибиться или написать не то. Данное задание не имеет какого-то “правильного” решения. Цель - достичь базового понимания проблемы. Чем больше вы фантазируете, тем лучше :) Тем не менее, старайтесь представить те ситуации, которые по-вашему мнению могли бы быть в реальных данных. Успеха!

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Представим фабрику по заготовке и продаже лекарственных трав.
У нас есть переменные:

high_workers  - рост сюорщиков трав;
productivity  - объем собранных трав на одного сборщика;
workers_count - количество сборщиков трав.

Продаже подлежит не весь собраный урожай. 
Поэтому вводим в дальнейшем поправку 0,9.



In [29]:
# Создаём сэмпл
n_samples = 1000

high_workers = np.random.choice(200, n_samples) + 150
productivity = np.random.choice(200, n_samples) + 10
workers_count = np.random.choice(250, n_samples) + 1

product = productivity * workers_count * 0.9

data = pd.DataFrame({'high_worker': high_worker, 'productivity': productivity, 'workers_count': workers_count, 'product': product})
data.head(5)

,high_worker,productivity,workers_count,product
0,312,168,171,25855.2
1,227,194,104,18158.4
2,300,171,244,37551.6
3,200,22,158,3128.4
4,182,29,156,4071.6


Обучаем модель и строим прогноз.

In [24]:
from sklearn.metrics import mean_absolute_error

X = data[['high_worker', 'productivity', 'workers_count']]
y = data['product']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['high_worker', 'productivity', 'workers_count']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [  0.35515875 114.38838607  96.38737802]
Bias: -12474.744411409434
Error: 2925.8554653806814


Ошибка достаточно высока: 2 926,0.

Поразмыслив логически, предполагаем, что рост рабочих не влияет на объем продаж.
Удаляем этот фактор из модели.

In [30]:
X = data[['productivity', 'workers_count']]
y = data['product']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['productivity', 'workers_count']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [114.67507857  99.85679664]
Bias: -12924.955882379578
Error: 2900.8909769278052


Ошибка немного снизилась. Но осталась на том же высоком уровне.

Анализируя дальше, понимаем, что можем еще сократить количество факторов. 

Весь собранный объем урожая представляет собой произведение количества работников на среднюю производительность каждого:

In [32]:
# Создаем новый признак
data['mult'] = data['productivity'] * data['workers_count']
data.head(5)

,high_worker,productivity,workers_count,product,mult
0,312,168,171,25855.2,28728
1,227,194,104,18158.4,20176
2,300,171,244,37551.6,41724
3,200,22,158,3128.4,3476
4,182,29,156,4071.6,4524


Строим модель по новому созданному признаку.

In [34]:
X = data[['mult']]
y = data['product']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['mult']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.9]
Bias: 7.275957614183426e-12
Error: 5.9671076968470515e-12


Ошибка значительно снизилась.